In [4]:
# Import dependencies 
import numpy as np
from twitter import *
import pandas as pd 
import datetime

In [5]:
#The tokens and consumers keys are my API authentication to use the Twitter API.  I'm using the twitter library as a python
#wrapper to call twitter request through python.
token = '1212463191910842368-l00ryMICeXQXXawl8w2zazEoFS7xDf'
token_secret = 'hbzjFRq1r2ygkydPXfrwrhm5zTa54F3mYEm2wi7Q09DgA'
consumer_key = 'g8V7jnF3dqMfg4LRhtobYB4Pl'
consumer_secret = 'Y0SHSL0H9X9gCtuy07mJ3cp144DS2JhwX4Uvgda2ph8NvIUswJ'

t = Twitter(
        auth=OAuth(token, token_secret, consumer_key, consumer_secret))

In [6]:
type(t)

twitter.api.Twitter

In [7]:
# The function below gets a person's timeline
# Note: Count can't be greater than 200
x = t.statuses.user_timeline(screen_name="BernieSanders", count=20, include_rts=False, tweet_mode = 'extended')

In [8]:
# Creating a for loop to inserts Hillary's tweets in a list
Bernie_tweet = []
for x in x:
    Bernie_tweet.append(x)

In [9]:
Bernie_tweet[0:5]

[{'created_at': 'Mon Jan 06 23:35:00 +0000 2020',
  'id': 1214329548998922243,
  'id_str': '1214329548998922243',
  'full_text': "This campaign has never been just about electing a president of the United States—as enormously important as that is.\n\nThis campaign is about transforming America.\n\nIt's about understanding that real change never takes place from the top on down, but always from the bottom on up.",
  'truncated': False,
  'display_text_range': [0, 280],
  'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
  'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 216776631,
   'id_str': '216776631',
   'name': 'Bernie Sanders',
   'screen_name': 'BernieSanders',
   'location': 'Vermont',
   'description': 'U.S. Senator f

In [10]:
type(Bernie_tweet)

list

In [11]:
# I just want to focus on ids and tweets
for element in Bernie_tweet:
    print (element['id'])
    print (element['full_text'])
    print('---')

1214329548998922243
This campaign has never been just about electing a president of the United States—as enormously important as that is.

This campaign is about transforming America.

It's about understanding that real change never takes place from the top on down, but always from the bottom on up.
---
1214316966141284353
If Congress wants to go to war — and I will vote against that — but if Congress wants to go to war, let Congress have the guts to vote for war. Do not let this president take unilateral action. https://t.co/8bGWByfAIB
---
1214307054724243456
Pundits and consultants may not understand our campaign.

But the people get it.

They're coming by the thousands to our rallies and town halls.

They're chipping in $5 at a time.

They understand that if we all do what we can, we can beat the billionaire class at their own game.
---
1214290326766149632
Congratulations to Sports Illustrated staff for organizing @si_union with @nyguild. I hope other workers in media follow their e

In [12]:
#(TweetMiner) I found this function that easily organizes the tweets to input them into a panda dataframe

class TweetMiner(object):

    
    def __init__(self, t, result_limit = 200):
        
        self.t = t        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="BernieSanders", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets, tweet_mode = 'extended')
                statuses = [x for x in statuses]
            else:
                statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets, tweet_mode = 'extended')
                statuses = [x for x in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [13]:
# Result limit == count parameter from our GetUserTimeline()
miner = TweetMiner(t, result_limit=200)

In [14]:
bernie = miner.mine_user_tweets(user="BernieSanders")
elizabeth = miner.mine_user_tweets(user="ewarren")

In [15]:
# each tweet is dictionary within a list
bernie[0:5]

[{'tweet_id': 1214329548998922243,
  'handle': 'BernieSanders',
  'retweet_count': 1251,
  'text': "This campaign has never been just about electing a president of the United States—as enormously important as that is.\n\nThis campaign is about transforming America.\n\nIt's about understanding that real change never takes place from the top on down, but always from the bottom on up.",
  'mined_at': datetime.datetime(2020, 1, 6, 22, 16, 27, 132030),
  'created_at': 'Mon Jan 06 23:35:00 +0000 2020'},
 {'tweet_id': 1214316966141284353,
  'handle': 'BernieSanders',
  'retweet_count': 1316,
  'text': 'If Congress wants to go to war — and I will vote against that — but if Congress wants to go to war, let Congress have the guts to vote for war. Do not let this president take unilateral action. https://t.co/8bGWByfAIB',
  'mined_at': datetime.datetime(2020, 1, 6, 22, 16, 27, 132030),
  'created_at': 'Mon Jan 06 22:45:00 +0000 2020'},
 {'tweet_id': 1214307054724243456,
  'handle': 'BernieSanders

In [16]:
for x in range(5):
    print (bernie[x]['text'])
    print('---')

This campaign has never been just about electing a president of the United States—as enormously important as that is.

This campaign is about transforming America.

It's about understanding that real change never takes place from the top on down, but always from the bottom on up.
---
If Congress wants to go to war — and I will vote against that — but if Congress wants to go to war, let Congress have the guts to vote for war. Do not let this president take unilateral action. https://t.co/8bGWByfAIB
---
Pundits and consultants may not understand our campaign.

But the people get it.

They're coming by the thousands to our rallies and town halls.

They're chipping in $5 at a time.

They understand that if we all do what we can, we can beat the billionaire class at their own game.
---
Congratulations to Sports Illustrated staff for organizing @si_union with @nyguild. I hope other workers in media follow their example and join a union. Now more than ever, we need a strong labor movement acr

In [17]:
for x in range(5):
    print (elizabeth[x]['text'])
    print('--')

I'll be speaking with @Maddow shortly about how to build a country and an economy that works for everyone—tune in on @MSNBC!
--
At town halls in Davenport, Iowa, and across the country, people tell me about what's broken—about the things they're afraid of. But in spite of the fear, they have hope that things can change, because they believe in America, and in each other. And I believe, too. https://t.co/i9BlIaFaGO
--
@RogerLau I hate to say “I told you so,” but... I told you so!
--
Together, we imagined a country where everyone has a fighting chance. Thank you, Dubuque! https://t.co/3zJXDiTJ4p
--
There's nothing I love more than having real conversations with voters about how to build the big, structural change we need. So in Maquoketa, Iowa, I went canvassing door-to-door!

If you're ready to knock doors, too, sign up for a canvass training! https://t.co/tLKQnjLFTe https://t.co/jRv4lrK3jZ
--


In [18]:
# Turning Bernie's tweet into a dataframe:
bernie_df = pd.DataFrame(bernie)
bernie_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Mon Jan 06 23:35:00 +0000 2020,BernieSanders,2020-01-06 22:16:27.132030,1251,This campaign has never been just about electi...,1214329548998922243
1,Mon Jan 06 22:45:00 +0000 2020,BernieSanders,2020-01-06 22:16:27.132030,1316,If Congress wants to go to war — and I will vo...,1214316966141284353
2,Mon Jan 06 22:05:37 +0000 2020,BernieSanders,2020-01-06 22:16:27.132030,1157,Pundits and consultants may not understand our...,1214307054724243456
3,Mon Jan 06 20:59:08 +0000 2020,BernieSanders,2020-01-06 22:16:27.132030,470,Congratulations to Sports Illustrated staff fo...,1214290326766149632
4,Mon Jan 06 20:03:40 +0000 2020,BernieSanders,2020-01-06 22:16:27.132030,1808,We need Medicare for All.\n\nBut to create a h...,1214276366948020225


In [19]:
# Doing the same for Elizabeth Warren
miner = TweetMiner(t, result_limit=200)
elizabeth_tweets = miner.mine_user_tweets("ewarren", max_pages=30)

In [20]:
elizabeth_df = pd.DataFrame(elizabeth_tweets)

In [21]:
elizabeth_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Tue Jan 07 02:11:23 +0000 2020,ewarren,2020-01-06 22:17:09.133673,431,I'll be speaking with @Maddow shortly about ho...,1214368906879684608
1,Tue Jan 07 01:55:28 +0000 2020,ewarren,2020-01-06 22:17:09.133673,121,"At town halls in Davenport, Iowa, and across t...",1214364900639809536
2,Tue Jan 07 01:21:38 +0000 2020,ewarren,2020-01-06 22:17:09.133673,27,"@RogerLau I hate to say “I told you so,” but.....",1214356383392522240
3,Tue Jan 07 00:26:26 +0000 2020,ewarren,2020-01-06 22:17:09.133673,131,"Together, we imagined a country where everyone...",1214342493455765504
4,Mon Jan 06 21:48:07 +0000 2020,ewarren,2020-01-06 22:17:09.133673,175,There's nothing I love more than having real c...,1214302652642811905


In [22]:
bernie_df.shape

(2281, 6)

In [23]:
elizabeth_df.shape

(2959, 6)

In [24]:
# Joined elizabeth & Bernie Dataframe
tweets = pd.concat([elizabeth_df, bernie_df], axis=0)
tweets.shape

(5240, 6)

In [25]:
tweets.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Tue Jan 07 02:11:23 +0000 2020,ewarren,2020-01-06 22:17:09.133673,431,I'll be speaking with @Maddow shortly about ho...,1214368906879684608
1,Tue Jan 07 01:55:28 +0000 2020,ewarren,2020-01-06 22:17:09.133673,121,"At town halls in Davenport, Iowa, and across t...",1214364900639809536
2,Tue Jan 07 01:21:38 +0000 2020,ewarren,2020-01-06 22:17:09.133673,27,"@RogerLau I hate to say “I told you so,” but.....",1214356383392522240
3,Tue Jan 07 00:26:26 +0000 2020,ewarren,2020-01-06 22:17:09.133673,131,"Together, we imagined a country where everyone...",1214342493455765504
4,Mon Jan 06 21:48:07 +0000 2020,ewarren,2020-01-06 22:17:09.133673,175,There's nothing I love more than having real c...,1214302652642811905


In [26]:
# The library below convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import TfidfVectorizer

# The library stores dictionary values as counts 
from collections import Counter

In [27]:
# Using TfidfVectorizer to find ngrams for Elizabeth's tweets
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [28]:
# Pulls all of Elizabeth's tweet text's into one giant string
summaries = "".join(elizabeth_df['text'])
summaries

'I\'ll be speaking with @Maddow shortly about how to build a country and an economy that works for everyone—tune in on @MSNBC!At town halls in Davenport, Iowa, and across the country, people tell me about what\'s broken—about the things they\'re afraid of. But in spite of the fear, they have hope that things can change, because they believe in America, and in each other. And I believe, too. https://t.co/i9BlIaFaGO@RogerLau I hate to say “I told you so,” but... I told you so!Together, we imagined a country where everyone has a fighting chance. Thank you, Dubuque! https://t.co/3zJXDiTJ4pThere\'s nothing I love more than having real conversations with voters about how to build the big, structural change we need. So in Maquoketa, Iowa, I went canvassing door-to-door!\n\nIf you\'re ready to knock doors, too, sign up for a canvass training! https://t.co/tLKQnjLFTe https://t.co/jRv4lrK3jZDonald Trump is dangerous. He\'s repeatedly undermined our diplomatic relationships for his own personal, 

In [29]:
# vect.builder_analyzer handles the preprocessing, tokenization and n-grams generation
ngrams_summaries = vect.build_analyzer()(summaries)
ngrams_summaries

['ll speaking',
 'speaking maddow',
 'maddow shortly',
 'shortly build',
 'build country',
 'country economy',
 'economy works',
 'works tune',
 'tune msnbc',
 'msnbc town',
 'town halls',
 'halls davenport',
 'davenport iowa',
 'iowa country',
 'country people',
 'people tell',
 'tell broken',
 'broken things',
 'things afraid',
 'afraid spite',
 'spite fear',
 'fear hope',
 'hope things',
 'things change',
 'change believe',
 'believe america',
 'america believe',
 'believe https',
 'https i9bliafago',
 'i9bliafago rogerlau',
 'rogerlau hate',
 'hate say',
 'say told',
 'told told',
 'told imagined',
 'imagined country',
 'country fighting',
 'fighting chance',
 'chance thank',
 'thank dubuque',
 'dubuque https',
 'https 3zjxditj4pthere',
 '3zjxditj4pthere love',
 'love having',
 'having real',
 'real conversations',
 'conversations voters',
 'voters build',
 'build big',
 'big structural',
 'structural change',
 'change need',
 'need maquoketa',
 'maquoketa iowa',
 'iowa went',
 'we

In [30]:
Counter(ngrams_summaries).most_common(20)

[('big structural', 214),
 ('structural change', 204),
 ('big structural change', 199),
 ('ve got', 181),
 ('health care', 136),
 ('got plan', 123),
 ('ve got plan', 123),
 ('climate change', 89),
 ('gun violence', 88),
 ('donald trump', 84),
 ('grassroots movement', 74),
 ('trump administration', 74),
 ('make sure', 71),
 ('ll fight', 70),
 ('student loan', 63),
 ('fight hard', 61),
 ('climate crisis', 59),
 ('loan debt', 59),
 ('student loan debt', 58),
 ('plan https', 57)]

In [31]:
#Doing the same process for Bernie
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('health care', 247),
 ('new deal', 103),
 ('green new', 99),
 ('green new deal', 99),
 ('donald trump', 86),
 ('fossil fuel', 85),
 ('working people', 81),
 ('white house', 76),
 ('young people', 65),
 ('health insurance', 62),
 ('student debt', 60),
 ('billionaire class', 56),
 ('american people', 56),
 ('climate change', 56),
 ('united states', 53),
 ('climate crisis', 53),
 ('wall street', 52),
 ('corporate greed', 52),
 ('working class', 51),
 ('join live', 48)]

In [32]:
#Wine search to look for #wine in the full text.  Twee_mode='extended' reflects the full tweet, not truncated.
wine = t.search.tweets(q='#wine', include_rts=False, tweet_mode='extended')

In [33]:
type(wine)

twitter.api.TwitterDictResponse

In [34]:
wine

{'statuses': [{'created_at': 'Tue Jan 07 03:17:45 +0000 2020',
   'id': 1214385607327305734,
   'id_str': '1214385607327305734',
   'full_text': 'RT @DemiCassiani: Tonight’s #wine for #dinner \n.\n#food @marquesderiscal @winewankers @winegal57 @always5star @lacsd @LisaRivera2207 @talkav…',
   'truncated': False,
   'display_text_range': [0, 140],
   'entities': {'hashtags': [{'text': 'wine', 'indices': [28, 33]},
     {'text': 'dinner', 'indices': [38, 45]},
     {'text': 'food', 'indices': [49, 54]}],
    'symbols': [],
    'user_mentions': [{'screen_name': 'DemiCassiani',
      'name': 'Demi Cassiani',
      'id': 252504967,
      'id_str': '252504967',
      'indices': [3, 16]},
     {'screen_name': 'marquesderiscal',
      'name': 'Marqués de Riscal',
      'id': 135437331,
      'id_str': '135437331',
      'indices': [55, 71]},
     {'screen_name': 'winewankers',
      'name': 'The Wine Wankers 🍾🤛🎉🥂',
      'id': 1969630836,
      'id_str': '1969630836',
      'indices': [72, 84]

In [35]:
wine_list = wine['statuses']

In [36]:
type(wine_list)

list

In [37]:
wine_list

[{'created_at': 'Tue Jan 07 03:17:45 +0000 2020',
  'id': 1214385607327305734,
  'id_str': '1214385607327305734',
  'full_text': 'RT @DemiCassiani: Tonight’s #wine for #dinner \n.\n#food @marquesderiscal @winewankers @winegal57 @always5star @lacsd @LisaRivera2207 @talkav…',
  'truncated': False,
  'display_text_range': [0, 140],
  'entities': {'hashtags': [{'text': 'wine', 'indices': [28, 33]},
    {'text': 'dinner', 'indices': [38, 45]},
    {'text': 'food', 'indices': [49, 54]}],
   'symbols': [],
   'user_mentions': [{'screen_name': 'DemiCassiani',
     'name': 'Demi Cassiani',
     'id': 252504967,
     'id_str': '252504967',
     'indices': [3, 16]},
    {'screen_name': 'marquesderiscal',
     'name': 'Marqués de Riscal',
     'id': 135437331,
     'id_str': '135437331',
     'indices': [55, 71]},
    {'screen_name': 'winewankers',
     'name': 'The Wine Wankers 🍾🤛🎉🥂',
     'id': 1969630836,
     'id_str': '1969630836',
     'indices': [72, 84]},
    {'screen_name': 'winegal57',
 

In [38]:
# Showing how #wine reflects within the full text
for element in wine_list[0:5]:
    print (element['id'])
    print (element['full_text'])
    print('---')

1214385607327305734
RT @DemiCassiani: Tonight’s #wine for #dinner 
.
#food @marquesderiscal @winewankers @winegal57 @always5star @lacsd @LisaRivera2207 @talkav…
---
1214385015221575681
Get a look at the behind the scenes of a #winery. Meet people in the #wine industry and learn about the grapes! Read our award winning blog https://t.co/OuXjJXzg5C
---
1214384910913290240
RT @kmarikmarikmari: メタリックボルドー＊痛くない落ちない＊イヤーカフのお気に入り登録数が10回を超えました！ #minne_fav https://t.co/IFkcyWGoKf
コーデを選ばないシンプルデザイン❣️
極細チェーンがお洒落です✨
箱入り無料…
---
1214384624002097152
RT @RetroCEO: An #artistic interpretation of a #grape #leaf, this would be a #cool #gift for a #wine loving friend. Or perhaps you enjoy wi…
---
1214384452224335874
RT @_FalconTours: #Wine @ The Inn at Little Washington https://t.co/G55iJgO3B6
---


In [39]:
#(TweetMiner) I found this function that easily organizes the tweets to input them into a panda dataframe

class TweetMiner(object):

    
    def __init__(self, t, result_limit = 200):
        
        self.t = t        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, topic="#wine", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.t.search.tweets(q='#wine', count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets, tweet_mode='extended')
                statuses = [x for x in statuses['statuses']]
            else:
                statuses   =   self.t.search.tweets(q='#wine', count=self.result_limit, include_rts=mine_retweets, tweet_mode='extended')
                statuses = [x for x in statuses['statuses']]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [40]:
miner = TweetMiner(t, result_limit=200)

In [41]:
wine = miner.mine_user_tweets(topic="#wine")

In [42]:
wine

[{'tweet_id': 1214385607327305734,
  'handle': 'KitchenSprout',
  'retweet_count': 42,
  'text': 'RT @DemiCassiani: Tonight’s #wine for #dinner \n.\n#food @marquesderiscal @winewankers @winegal57 @always5star @lacsd @LisaRivera2207 @talkav…',
  'mined_at': datetime.datetime(2020, 1, 6, 22, 18, 16, 28257),
  'created_at': 'Tue Jan 07 03:17:45 +0000 2020'},
 {'tweet_id': 1214385015221575681,
  'handle': 'Dracaenawines',
  'retweet_count': 0,
  'text': 'Get a look at the behind the scenes of a #winery. Meet people in the #wine industry and learn about the grapes! Read our award winning blog https://t.co/OuXjJXzg5C',
  'mined_at': datetime.datetime(2020, 1, 6, 22, 18, 16, 28257),
  'created_at': 'Tue Jan 07 03:15:24 +0000 2020'},
 {'tweet_id': 1214384910913290240,
  'handle': 'atelierlaperle',
  'retweet_count': 10,
  'text': 'RT @kmarikmarikmari: メタリックボルドー＊痛くない落ちない＊イヤーカフのお気に入り登録数が10回を超えました！ #minne_fav https://t.co/IFkcyWGoKf\nコーデを選ばないシンプルデザイン❣️\n極細チェーンがお洒落です✨\n箱入り無料…',
  'mined_at': datet

In [43]:
wine_tweet_df = pd.DataFrame(wine)
wine_tweet_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Tue Jan 07 03:17:45 +0000 2020,KitchenSprout,2020-01-06 22:18:16.028257,42,RT @DemiCassiani: Tonight’s #wine for #dinner ...,1214385607327305734
1,Tue Jan 07 03:15:24 +0000 2020,Dracaenawines,2020-01-06 22:18:16.028257,0,Get a look at the behind the scenes of a #wine...,1214385015221575681
2,Tue Jan 07 03:14:59 +0000 2020,atelierlaperle,2020-01-06 22:18:16.028257,10,RT @kmarikmarikmari: メタリックボルドー＊痛くない落ちない＊イヤーカフの...,1214384910913290240
3,Tue Jan 07 03:13:51 +0000 2020,kellybaugher16,2020-01-06 22:18:16.028257,32,RT @RetroCEO: An #artistic interpretation of a...,1214384624002097152
4,Tue Jan 07 03:13:10 +0000 2020,_FalconTours,2020-01-06 22:18:16.028257,1,RT @_FalconTours: #Wine @ The Inn at Little Wa...,1214384452224335874


In [82]:
wine_list = (wine_tweet_df.text)

In [83]:
type(wine_list)

pandas.core.series.Series

In [44]:
#Vectorizing the wine df
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(wine_tweet_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('wine https', 230),
 ('wine winelover', 82),
 ('food wine', 50),
 ('white wine', 46),
 ('wine vino', 42),
 ('georges duboeuf', 41),
 ('red wine', 40),
 ('beer wine', 39),
 ('vino wine', 35),
 ('2020 https', 30),
 ('vin wine', 29),
 ('winelover https', 26),
 ('wine vin', 26),
 ('wine winelovers', 25),
 ('wine beer', 24),
 ('winemonday https', 24),
 ('beaujolais wine', 22),
 ('amp wine', 21),
 ('new year', 21),
 ('sonoma county', 20)]

In [52]:
from textblob import TextBlob

In [91]:
# Showing tweet sentiment in Bernie df
# Polarity ranges from -1 (most negative) to 1 (most positive).
# Subjectivity ranges from 0.0 (very objective) to 1.0 (very subjective)
for tweet in bernie_df['text']:
    print(tweet)
    analysis = TextBlob(tweet)
    print('---')
    print(analysis.sentiment)
    print('---')

This campaign has never been just about electing a president of the United States—as enormously important as that is.

This campaign is about transforming America.

It's about understanding that real change never takes place from the top on down, but always from the bottom on up.
---
Sentiment(polarity=0.23611111111111113, subjectivity=0.5222222222222223)
---
If Congress wants to go to war — and I will vote against that — but if Congress wants to go to war, let Congress have the guts to vote for war. Do not let this president take unilateral action. https://t.co/8bGWByfAIB
---
Sentiment(polarity=6.938893903907228e-18, subjectivity=0.24999999999999997)
---
Pundits and consultants may not understand our campaign.

But the people get it.

They're coming by the thousands to our rallies and town halls.

They're chipping in $5 at a time.

They understand that if we all do what we can, we can beat the billionaire class at their own game.
---
Sentiment(polarity=0.09999999999999998, subjectivit

Sentiment(polarity=0.04545454545454545, subjectivity=0.19999999999999998)
---
Wall Street’s greed has left millions unbanked or underbanked. We must establish basic banking services at post offices across the country. https://t.co/ZoLqKQjd4T
---
Sentiment(polarity=0.0, subjectivity=0.0625)
---
In 1979 students paid $500 a year in today's dollars to attend Cal State University.

Today they pay $7,300.

We're failing our young people, but it's not too late to fix.

We're going to make public colleges, universities and trade schools tuition-free and cancel student debt.
---
Sentiment(polarity=-0.06666666666666667, subjectivity=0.35555555555555557)
---
Unlike Trump, we will not abandon fossil fuel workers.

Our Green New Deal guarantees:
-5 years of a fossil fuel worker's previous salary
-Housing assistance
-Job training
-Health care
-Pension and early retirement support
-Priority placement in new good-paying union jobs https://t.co/ne5TdQQMNG
---
Sentiment(polarity=0.0012121212121212089, 

Sentiment(polarity=0.35000000000000003, subjectivity=0.5499999999999999)
---
McDonald's claims it can't afford to pay a living wage to all workers.

Now it's giving its former CEO, who was fired, a $42 million exit package.

Corporate greed has no limit and we need a president who is prepared to take them on and end it.

That is exactly what I plan to do.
---
Sentiment(polarity=0.08333333333333333, subjectivity=0.08333333333333333)
---
Greedy, corrupt fossil fuel corporations created the climate crisis. We will make them pay to stop it.
---
Sentiment(polarity=-0.5, subjectivity=1.0)
---
The Green New Deal is not just a climate plan. It is an economic plan. We will create 20 million jobs and invest $16.3 trillion in our economy.
---
Sentiment(polarity=0.04545454545454545, subjectivity=0.3181818181818182)
---
The billionaire class is scared and they should be scared.
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
General Mills made over $1.7 billion in profits last year. They should b

---
Pharma executives knew opioids were addicting and killing people. They sold them anyway.

Fossil fuel executives knew their emissions were threatening the planet. They continued anyway.

This is what unfettered capitalism is doing to our world.

We are going to end this greed.
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
This is a positive step forward in these negotiations. Our movement continues to stand with the thousands of @UAW members who are still walking picket lines across the country. https://t.co/8cJKG7aYA4
---
Sentiment(polarity=0.22727272727272727, subjectivity=0.5454545454545454)
---
This is the difference between myself and Joe Biden. https://t.co/DVajgDHdCZ
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
When we win, we are going to fundamentally change the nature of the presidency. I will not only be commander-in-chief, but organizer-in-chief. And we will take on the most powerful corporations and special interests.
---
Sentiment(polarity=0.39142857142857146

This is astonishing.

1 million donors have given to our campaign, faster than any in history. We've now received more than 2.5 million contributions.

We are the only campaign with more support than Trump. We will defeat him and build a country that works for all of us. https://t.co/y76kfWACP2
---
Sentiment(polarity=0.375, subjectivity=0.75)
---
This campaign condemns racist bullying and harassment of any kind, in any space. We are building a multiracial movement for justice — that’s how we win the White House.
---
Sentiment(polarity=0.4666666666666666, subjectivity=0.43333333333333335)
---
Remember four years ago when we talked about making public colleges and universities tuition-free, and people called us "radical and extreme"? Not anymore.

When we stand together and have a bold vision for the future, real change can happen. https://t.co/9SzbDFJqFl
---
Sentiment(polarity=0.08166666666666667, subjectivity=0.43166666666666664)
---
We cannot transform this country without getting mil

Sentiment(polarity=0.13333333333333333, subjectivity=0.43333333333333335)
---
This campaign is not about middle ground solutions. 

This campaign is about transforming our country so that it finally works for all people, not just the rich and powerful.

We are not afraid to be bold. https://t.co/PMwZhCOM5F
---
Sentiment(polarity=0.21805555555555556, subjectivity=0.7194444444444444)
---
The #GreenNewDeal will:
✅ Create 20 million jobs
✅ Avoid climate catastrophe
✅ Make fossil fuel companies pay their fair share
✅ Prioritize the most impacted communities

This is the healthy and habitable future our people deserve. https://t.co/wB5UU5uJqt
---
Sentiment(polarity=0.425, subjectivity=0.50625)
---
We are the strongest campaign to defeat Donald Trump, and we'll do it by bringing millions of people into the political process, in every corner of this country, and build a movement based on justice. Thank you, Sacramento! https://t.co/BuPStJn7Ir
---
Sentiment(polarity=0.0, subjectivity=0.1)
---
W

---
Sentiment(polarity=0.2833333333333334, subjectivity=0.7333333333333334)
---
The pharmaceutical industry is the biggest bunch of crooks in this country.

When I am president, we are going to end their greed and slash prices for prescription drugs. https://t.co/HlLmP5b5IQ
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
Rally with Bernie in Santa Monica --&gt; https://t.co/9hyrt6XsuZ
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
Donald Trump said he'd be a different kind of Republican. Instead, he has continued the same attacks on Medicare, Medicaid and Social Security.

We are not going to abandon our seniors. Every American has a right to a secure retirement. https://t.co/SgNw27Spar
---
Sentiment(polarity=0.18843537414965986, subjectivity=0.40391156462585037)
---
A vial of insulin costs $300 in the United States.

On Sunday, I'm joining a group of diabetics to go across the border to Canada, where the same medicine — the exact same — costs $30. https://t.co/PaNKlu3XIJ
---
Sent

Sentiment(polarity=0.039285714285714285, subjectivity=0.6928571428571428)
---
It is imperative that we remain steadfast in our commitment to guarantee health care as a human right and no longer allow private corporations to make billions of dollars in profits off Americans’ health care.
---
Sentiment(polarity=0.17142857142857143, subjectivity=0.45267857142857143)
---
If you support Medicare for All, you must be willing to end the greed of the health insurance and pharmaceutical industries. That means transforming our dysfunctional system by ending the use of private health insurance, except to cover non-essential care like cosmetic surgeries.
---
Sentiment(polarity=0.125, subjectivity=0.5625)
---
Congratulations to Illinois for legalizing marijuana and expunging 770,000 marijuana-related cases. It is time to legalize marijuana nationwide. https://t.co/WX1Ic2iy8P
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
We must have the guts to take on:

-Wall Street
-The insurance industry
-Th

In [89]:
# Showing tweet sentiment in wine
for tweet in wine_tweet_df['text']:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)

RT @DemiCassiani: Tonight’s #wine for #dinner 
.
#food @marquesderiscal @winewankers @winegal57 @always5star @lacsd @LisaRivera2207 @talkav…
Sentiment(polarity=0.0, subjectivity=0.0)
Get a look at the behind the scenes of a #winery. Meet people in the #wine industry and learn about the grapes! Read our award winning blog https://t.co/OuXjJXzg5C
Sentiment(polarity=0.0, subjectivity=0.725)
RT @kmarikmarikmari: メタリックボルドー＊痛くない落ちない＊イヤーカフのお気に入り登録数が10回を超えました！ #minne_fav https://t.co/IFkcyWGoKf
コーデを選ばないシンプルデザイン❣️
極細チェーンがお洒落です✨
箱入り無料…
Sentiment(polarity=0.0, subjectivity=0.0)
RT @RetroCEO: An #artistic interpretation of a #grape #leaf, this would be a #cool #gift for a #wine loving friend. Or perhaps you enjoy wi…
Sentiment(polarity=0.4208333333333333, subjectivity=0.7749999999999999)
RT @_FalconTours: #Wine @ The Inn at Little Washington https://t.co/G55iJgO3B6
Sentiment(polarity=-0.1875, subjectivity=0.5)
RT @winewankers: A great place to live! #wine #love #laughter https://t.co/uKtpYQH8r5
Se

Sentiment(polarity=0.0, subjectivity=0.0)
#Wine Fact:  Spanish wines may only be labelled Crianza, Reserva or Gran Reserva if they have spent minimum times maturing in barrel and bottle before release. 🍷
Sentiment(polarity=-0.03333333333333333, subjectivity=0.3666666666666667)
Chandon

Lujan de Cuyo, Mendoza, Argentina

Post completo com a visita disponível no blog.

https://t.co/coB9MRFSgJ

#Vinho #Wine #WineLovers #Vin #Vino #Vinícola #Winery #Chandon #Argentina #Mendoza #LujandeCuyo #Agrelo #Cave #SparklingWine #Espumantes #Chardonnay #PinotNoir
Sentiment(polarity=0.0, subjectivity=0.0)
RT @Julianna_glass: The moment so peaceful, 
so beautiful. 
The moment of quiet thoughts 
and reflections.

The moment of #SundaySunsets wi…
Sentiment(polarity=0.3666666666666667, subjectivity=0.611111111111111)
RT @anniefoss: “I’ve made the gravy mum, &amp; used the wine with the pretty label!” Half a bottle of 2001 chateauneuf du pape, in with scrag e…
Sentiment(polarity=0.07291666666666667, subjec

Sentiment(polarity=0.0, subjectivity=0.0)
My number 3..top white #wine From #moldova. .interestingly Moldovan wineries are winning most of their medals in competitions for this varietal  Viorica. .unique to Moldova https://t.co/57xHbfPtzy
Sentiment(polarity=0.3333333333333333, subjectivity=0.4166666666666667)
Blue and White Outfits for Spring https://t.co/qt5vcjwOcs #ttot #travel #wanderlust #wine
Sentiment(polarity=0.0, subjectivity=0.05)
Classic Journeys Review: Luxury Touring? Oh Yes. https://t.co/BeaLVMsSiR #ttot #travel #wanderlust #wine
Sentiment(polarity=0.16666666666666666, subjectivity=0.16666666666666666)
Winter Whites to Put on Your Radar https://t.co/fGO2Ya5XV5 #ttot #travel #wanderlust #wine
Sentiment(polarity=0.0, subjectivity=0.0)
6 Places to Visit a Winter Festival in North America https://t.co/eBRrq8KrX8 #ttot #travel #wanderlust #wine
Sentiment(polarity=0.0, subjectivity=0.0)
RT @RoebuckSteve1: Great #culinary #finefood &amp; #finewine experience with some #winelover 

Sentiment(polarity=0.46875, subjectivity=0.625)
RT @re_hungry: Happy #Sunday evening lovely Hungrys! Lasagne is ready, just topping up the wine &amp; about to indulge in a slice. Have a fabul…
Sentiment(polarity=0.5416666666666666, subjectivity=0.75)
#Uganda police detain #Bobi #Wine, foil meeting with supporters https://t.co/tOQE6nlle9
Sentiment(polarity=0.0, subjectivity=0.0)
RT @alawine: ...Here's a handy chart to check if you need #wine today! ;) RT @LoireValleyWine https://t.co/37e2UKG5UE
Sentiment(polarity=0.5, subjectivity=0.95)
RT @alawine: These #wine glasses float in the water! #ocean #bath #pool #spa SOLD! RT @smartwomancoach #wine #winelover https://t.co/BB0Fzs…
Sentiment(polarity=0.0, subjectivity=0.0)
El Pastel de Pescado de  lajaranataberna , el mítico Pintxo del Astelena     #wine #vino #cocinatradicional #bar #bares #pinchos #sansebastian #cocinero #blogcocinaresunsentimiento #instagramers #restaurant… https://t.co/N5t7qQor1w
Sentiment(polarity=0.0, subjectivity=0.0)
5

Sentiment(polarity=0.0, subjectivity=0.0)
All’Epifania Il colpo di grazia. La Cassoeula. Si parte. #cucina #freddo #wine #Lombardia https://t.co/8DL5gIbQ0a
Sentiment(polarity=0.0, subjectivity=0.0)
RT @TheVinum: If you come on vacation in #Abruzzo and visit our wine bar, we will offer you a tasting of our #organic rosé #wine Cerasuolo…
Sentiment(polarity=0.0, subjectivity=0.0)
